*********************JUAN CARLOS PAULINO FONSECA GOMEZ*************************

# Capstone Project Stored Procedures: 
GetMaxQuantity, ManageBooking, UpdateBooking, AddBooking, CancelBooking

#Establishing a connection and cursor

In [3]:
import mysql.connector as connector

In [14]:
connection=connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="Armin12B",
    database="capstone"
)

cursor=connection.cursor()

db_Info = connection.get_server_info()
print("Connected to MySQL Server version ", db_Info)

Connected to MySQL Server version  8.0.31


#GetMaxQuantity

In [5]:
cursor.execute("""
    CREATE PROCEDURE GetMaxQuantity ()
    BEGIN
        SELECT MAX(OrderQty)
        FROM `order`;
    END""")
cursor.callproc('GetMaxQuantity')
for result in cursor.stored_results():
    print(result.fetchall())

[(4,)]


#ManageBooking

In [60]:
cursor.execute("""
    CREATE PROCEDURE ManageBooking (
        IN p_BookingsDate DATE,
        IN p_BookingsTable INT
    )
    BEGIN
        DECLARE BookingsCount INT;
        
        SELECT COUNT(*) INTO BookingsCount
        FROM bookings
        WHERE BookingsDate = p_BookingsDate AND BookingsTable = p_BookingsTable;
        
        IF BookingsCount > 0 THEN
            SELECT CONCAT('Table ', p_BookingsTable, ' is already booked') AS message;
        ELSE
            SELECT CONCAT('Table ', p_BookingsTable, ' is available') AS message;
        END IF;
    END""")

In [61]:
cursor.callproc('ManageBooking', ('2023-03-15', 1))
for result in cursor.stored_results():
    print(result.fetchall())

[('Table 1 is available',)]


In [62]:
cursor.callproc('ManageBooking', ('2023-03-20', 3))
for result in cursor.stored_results():
    print(result.fetchall())

[('Table 3 is already booked',)]


#UpdateBooking

In [5]:
cursor.execute("""
    CREATE PROCEDURE UpdateBooking(
        IN p_booking_id INT,
        IN p_booking_table INT,
        IN p_booking_date DATE
    )
    BEGIN
        UPDATE bookings
        SET BookingsTable = p_booking_table, BookingsDate = p_booking_date    
        WHERE idBookings = p_booking_id;
    END""")

In [8]:
cursor.callproc('UpdateBooking', (1, 3, '2022-11-23'))

(1, 3, '2022-11-23')

#AddBooking

In [15]:
cursor.execute("""
    CREATE PROCEDURE AddBooking (
        IN table_number INT,
        IN booking_date DATE
    )
    BEGIN
        DECLARE booking_status BOOL;
        SELECT COUNT(*) INTO booking_status
        FROM bookings
        WHERE BookingsTable = table_number AND BookingsDate = booking_date;
        IF booking_status = 0 THEN
            INSERT INTO bookings (BookingsTable, BookingsDate)
            VALUES (table_number, booking_date);
            SELECT CONCAT('Booking for table ', table_number, ' on ', booking_date, ' was successful') AS message;
        ELSE
            SELECT CONCAT('Table ', table_number, ' is already booked on ', booking_date) AS message;
        END IF;
    END""")

In [ ]:
cursor.callproc('AddBooking', (1, '2023-03-15'))
for result in cursor.stored_results():
    print(result.fetchall())

#CancelBooking

In [19]:
cursor.execute("""
    CREATE PROCEDURE CancelBooking(IN booking_id INT)
    BEGIN
        DELETE FROM bookings WHERE BookingID = booking_id;
    END""")

In [ ]:
cursor.callproc('CancelBooking', (1,))

In [21]:
cursor.close()

True